<a href="https://colab.research.google.com/github/jehbs/Code-sumo/blob/master/precoImoveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




```
# importar algumas bibliotecas, entre elas a google colab para ler o arquivo do drive
```





In [5]:
import pandas as pd

#from xgboost import XGBClassifier


import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn.tree import DecisionTreeRegressor


from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
from google.colab import files
uploaded = files.upload() 

Saving Precos_Imoveis.csv to Precos_Imoveis.csv




```
#Ler o arquivo e tirar os nulos transformando em 0 
```



In [0]:
import pandas as pd
import io
data = pd.read_csv(io.StringIO(uploaded['Precos_Imoveis.csv'].decode('utf-8')))
#data = pd.read_csv('Precos_Imoveis.csv')
data = data.fillna(0)

In [13]:
data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,57.623288,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.117123,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1868.739726,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,34.664304,9981.264932,1.382997,1.112799,30.202904,20.645407,180.731373,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,453.697295,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,0.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,42.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1958.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,63.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1977.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,79.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,164.250000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2001.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [0]:
data



```
# Aplicar uma tecnica para transformar todas as variaveis categóricas em numéricas 
```



In [0]:
one_hot_enco = ['MSZoning', 'LotFrontage', 'Street',
                'Alley','LotShape','LandContour','Utilities',
                'LotConfig', 'LandSlope','Neighborhood', 'Condition1', 
                'Condition2','BldgType', 'HouseStyle', 'RoofStyle',
                'RoofMatl','Exterior1st','Exterior2nd', 'MasVnrType', 
                'ExterQual', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
                'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir',
                'Electrical', 'KitchenQual', 'Functional', 'Fireplaces', 
                'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 
                'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
                'SaleType', 'SaleCondition', 'ExterCond', 'FireplaceQu'     
               ]
for i in one_hot_enco:
  novas_colunas = np.array(data[i])
  encoded = pd.get_dummies(novas_colunas,dummy_na=True)
  encoded = encoded.add_prefix(i)
  data = pd.concat([data,encoded],axis=1)
  data = data.drop([i], axis=1)

Criação dos DataFrame X e Y 

In [0]:
Y = data['SalePrice']
X = data.drop(['SalePrice', 'Id'], axis=1)
X = X.loc[:,~X.columns.duplicated()]

In [16]:
X.describe()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoningC (all),MSZoningFV,MSZoningRH,MSZoningRL,MSZoningRM,MSZoningnan,...,FenceGdWo,FenceMnPrv,FenceMnWw,Fencenan,MiscFeature0,MiscFeatureGar2,MiscFeatureOthr,MiscFeatureShed,MiscFeatureTenC,MiscFeaturenan,SaleTypeCOD,SaleTypeCWD,SaleTypeCon,SaleTypeConLD,SaleTypeConLI,SaleTypeConLw,SaleTypeNew,SaleTypeOth,SaleTypeWD,SaleTypenan,SaleConditionAbnorml,SaleConditionAdjLand,SaleConditionAlloca,SaleConditionFamily,SaleConditionNormal,SaleConditionPartial,SaleConditionnan,ExterCondEx,ExterCondFa,ExterCondGd,ExterCondPo,ExterCondTA,ExterCondnan,FireplaceQu0,FireplaceQuEx,FireplaceQuFa,FireplaceQuGd,FireplaceQuPo,FireplaceQuTA,FireplaceQunan
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.0,...,1460.000000,1460.000000,1460.000000,1460.0,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.0,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.0,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.0,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.0,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.0
mean,56.897260,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.117123,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,1868.739726,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,0.006849,0.044521,0.010959,0.788356,0.149315,0.0,...,0.036986,0.107534,0.007534,0.0,0.963014,0.001370,0.001370,0.033562,0.000685,0.0,0.029452,0.002740,0.001370,0.006164,0.003425,0.003425,0.083562,0.002055,0.867808,0.0,0.069178,0.002740,0.008219,0.013699,0.820548,0.085616,0.0,0.002055,0.019178,0.100000,0.000685,0.878082,0.0,0.472603,0.016438,0.022603,0.260274,0.013699,0.214384,0.0
std,42.300571,9981.264932,1.382997,1.112799,30.202904,20.645407,180.731373,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,453.697295,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,0.082505,0.206319,0.104145,0.408614,0.356521,0.0,...,0.188793,0.309897,0.086502,0.0,0.188793,0.036999,0.036999,0.180160,0.026171,0.0,0.169128,0.052289,0.036999,0.078298,0.058440,0.058440,0.276824,0.045299,0.338815,0.0,0.253844,0.052289,0.090317,0.116277,0.383862,0.279893,0.0,0.045299,0.137198,0.300103,0.026171,0.327303,0.0,0.499420,0.127198,0.148684,0.438934,0.116277,0.410535,0.0
min,20.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000

In [0]:
from sklearn import preprocessing
array = X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(array)
X = pd.DataFrame(x_scaled)

In [0]:
Y.describe()

# Inicio do Modelo realmente 



```
# Import das funções de métrica R² no caso
```



In [0]:
from sklearn.metrics import r2_score

def performance_metric(y_true, y_predict):
    
    score = r2_score(y_true, y_predict)

    return score



```
# separação do conjunto de treino e teste
```



In [0]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state = 42)



```
# Criação da função com o GridSearch
```



In [0]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

def fit_model(X, y):

    cv_sets = ShuffleSplit(n_splits = 10, test_size = 0.20, random_state = 0)
    
    regressor = DecisionTreeRegressor()

    params = {'max_depth':[1,2,3,4,5,6,7,8,9,10,11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]}
    
    scoring_fnc = make_scorer(performance_metric)

    grid = GridSearchCV(estimator=regressor, param_grid=params, scoring=scoring_fnc, cv=cv_sets)

    grid = grid.fit(X, y)

    return grid.best_estimator_


Aplicação do modelo Sem chamar o Grid

In [22]:
regressao_inicial = DecisionTreeRegressor()
regressao_inicial.fit(X_train, y_train)
Y_pred_inicial = regressao_inicial.predict(X_test)
performace_inicial = performance_metric(y_test,Y_pred_inicial)
performace_inicial

0.7972549404388496

Aplicação com o Grid

In [23]:
# Fit the training data to the model using grid search
reg = fit_model(X_train, y_train)

# Produce the value for 'max_depth'
print("Parameter 'max_depth' is {} for the optimal model.".format(reg.get_params()['max_depth']))

Y_prediction = reg.predict(X_test)

performace = performance_metric(y_test,Y_prediction)
performace

Parameter 'max_depth' is 19 for the optimal model.


0.7624970634310273

Analise das Features mais importantes

In [24]:
feat_importances = pd.Series(reg.feature_importances_, index=X.columns)
feat_importances.head(20)

0     4.737413e-04
1     1.760861e-02
2     5.713967e-01
3     5.868590e-03
4     6.989421e-03
5     7.555599e-03
6     3.137545e-03
7     2.032786e-02
8     6.852145e-04
9     6.890889e-03
10    3.222471e-02
11    1.507835e-02
12    5.769971e-02
13    0.000000e+00
14    1.173339e-01
15    9.360472e-05
16    4.300125e-07
17    7.949908e-05
18    1.589559e-04
19    2.961583e-03
dtype: float64

In [0]:
feat_importances.nlargest(15).plot(kind='barh')
plt.show()

Aplicação do Modelo apenas com as features mais importantes

In [0]:
 X_new = X[feat_importances.nlargest(15).index]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_new, Y, test_size=0.3, random_state = 42)

In [0]:
reg_new = fit_model(X_train, y_train)
Y_prediction_new = reg_new.predict(X_test)

In [0]:
performace_new = performance_metric(y_test,Y_prediction)
performace_new